In [5]:
"""
Pull year-by-year Statcast pitching data for Shohei Ohtani using the pybaseball API
Covers seasons in which he pitched (2018, 2021–2023), skipping injured years
Appends all years into a single DataFrame and exports it as a CSV for downstream processing and Power BI visualization
"""

# Import pybaseball for Statcast API access, pandas for data handling
from pybaseball import statcast_pitcher
import pandas as pd

# Ohtani's MLBAM ID
ohtani_id = 660271

# Define only the seasons where he actually pitched (skip 2019 & 2020 due to injury)
years = [2018, 2021, 2022, 2023]

# Pull and append all yearly data into one list of DataFrames
dfs = []
for year in years:
    # Statcast season runs roughly Mar–Oct
    data = statcast_pitcher(f'{year}-03-01', f'{year}-10-01', ohtani_id)
    data['year'] = year  # Add a clean year column since game_date is messy for slicing
    dfs.append(data)

# Combine all years into a single DataFrame
ohtani_pitching = pd.concat(dfs, ignore_index=True)

# Save full raw export to local CSV (keep this version for archival or future reprocessing)
ohtani_pitching.to_csv("ohtani_pitching_2018_2023.csv", index=False)

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


In [8]:
"""
Load the full pitching dataset into DuckDB and run a SQL query to isolate only the relevant columns
Extracts year, pitch type, velocity, spin rate, zone, and outcomes for each pitch
Exports a cleaned CSV for direct use in Tableau for YoY pitch type analysis
"""

import duckdb

# Load your CSV file into a DuckDB virtual table
con = duckdb.connect()

# Run SQL query to keep only the relevant columns
result = con.execute("""
    SELECT
        year AS "Year",
        pitch_name AS "Pitch Type",
        release_speed AS "Velocity",
        description AS "Pitch Outcome",
        zone AS "Zone",
        release_spin_rate AS "Spin Rate",
        events AS "Plate Appearance Outcome"
    FROM 'ohtani_pitching_2018_2023.csv'
""").fetchdf()

result.to_csv("ohtani_pitching_cleaned.csv", index=False)